Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [4]:
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
len(data["target"])

857

In [6]:
data.target.shape

(857,)

In [7]:
data["target"]

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
# Create Pipeline Components
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

#Scikit learn pipeline helpful
rfc = RandomForestClassifier()

In [12]:
# Define the Pipeline

# (from sklearn.pipeline import Pipeline
# pipeline makes code cleaner and more concise, less boilerplate
# serialize and save to disk - preprocessing, transforming, datacleaning)

# from sklearn.pipeline import make_pipeline - functionally identical, with pipe can name without having to change
# hyperparameter tuning all at once

pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

<img src https://miro.medium.com/proxy/1*ZTlQm_WRcrNqL-nLnx6GJA.png /img>

In [13]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  3.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

https://scikit-learn.org/stable/_images/grid_search_cross_validation.png

In [14]:
grid_search.best_score_

0.8844553243574053

In [15]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [16]:
grid_search.predict(['Send me lots of money now', 'you wond the lottery in Nigeria'])

array([1, 1])

In [18]:
grid_search.predict_proba(['Send me lots of money now', 'you wond the lottery in Nigeria'])

array([[0.20547649, 0.79452351],
       [0.20547649, 0.79452351]])

In [20]:
categories1 = ['talk.politics.misc',
              'sci.space']

data = fetch_20newsgroups(subset='train', categories=categories1, remove = ('headers', 'footers', 'quotes'))

In [22]:
data["target"].shape

(1058,)

In [24]:
data["data"][2]

"\nMy opinion is this:  In a society whose economy is primarily based on \ncapitalism, the role of government should be to provide those goods and \nservices that need providing for the general public's good.  BUT government \nshould supply those necessary goods and services only when it is impossible \nfor a private enterprise (or individual) to make money from providing them.\nI agree with some of the other posts that this train probably can not make \nmoney and will rely heavily on State tax dollars.  \n\nThe question, I think, then becomes:  Do we, the general public, need the train?\n\nI certainly do not, nor will I ever, need this train in Lubbock, Texas.  With\nthe inexpensive air travel provided between Dallas and Houston, I don't think\npeople in Dallas or Houston need it either."

In [25]:
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [28]:
grid_search.best_score_

0.8307878029151391

In [29]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [30]:
grid_search.predict(['This is a free society', 
                     'Elections will be in November', 
                     "Covid doesn't care what your political beliefs are.", 
                     'The team successfully launched their satellites into low-earth orbits',
                     'The information paradox was solved last week, say some scientists.',
                     'We will be living on Mars in 2050'])

array([1, 0, 1, 0, 0, 0])

In [31]:
grid_search.predict_proba(['This is a free society', 
                     'Elections will be in November', 
                     "Covid doesn't care what your political beliefs are.", 
                     'The team successfully launched their satellites into low-earth orbits',
                     'The information paradox was solved last week, say some scientists.',
                     'We will be living on Mars in 2050'])

array([[0.45066789, 0.54933211],
       [0.64944616, 0.35055384],
       [0.46136054, 0.53863946],
       [0.6816457 , 0.3183543 ],
       [0.53685138, 0.46314862],
       [0.64944616, 0.35055384]])

In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
import scipy

parameters = {
    'vect__max_df': scipy.stats.uniform(0.8, 0.99),
    'vect__min_df': scipy.stats.uniform(.01, .05),
    'vect__max_features': scipy.stats.randint(500,1000),
    'clf__n_estimators':scipy.stats.randint(30, 100),
    'clf__max_depth':scipy.stats.randint(20,100)
}

rand_search = RandomizedSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1, n_iter=25)
rand_search.fit(data.data, data.target)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   36.1s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [34]:
rand_search.best_score_

0.8497000815520646

In [35]:
rand_search.best_params_

{'clf__max_depth': 44,
 'clf__n_estimators': 39,
 'vect__max_df': 1.2113543678637089,
 'vect__max_features': 540,
 'vect__min_df': 0.022222477982974862}

In [36]:
rand_search.predict(['This is a free society', 
                     'Elections will be in November', 
                     "Covid doesn't care what your political beliefs are.", 
                     'The team successfully launched their satellites into low-earth orbits',
                     'The information paradox was solved last week, say some scientists.',
                     'We will be living on Mars in 2050'])

array([1, 0, 0, 0, 0, 0])

In [37]:
rand_search.predict_proba(['This is a free society', 
                         'Elections will be in November', 
                         "Covid doesn't care what your political beliefs are.", 
                         'The team successfully launched their satellites into low-earth orbits',
                         'The information paradox was solved last week, say some scientists.',
                         'We will be living on Mars in 2050'])

array([[0.49084647, 0.50915353],
       [0.71918705, 0.28081295],
       [0.5303031 , 0.4696969 ],
       [0.80960626, 0.19039374],
       [0.59006493, 0.40993507],
       [0.71918705, 0.28081295]])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

## Challenge

You're trying to achieve 75% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

https://research.fb.com/wp-content/uploads/2016/11/post00049_image0001.png

In [44]:
# Import
from sklearn.decomposition import TruncatedSVD

#Singular Value Decomposition
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [45]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
#pipeline inside a pipeline
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [46]:
params = { 
    'lsi__svd__n_components': scipy.stats.randint(10, 100),
    'lsi__vect__max_df': scipy.stats.uniform(.80, .99),
    'clf__n_estimators': scipy.stats.randint(10, 100)
}

In [47]:
print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

In [48]:
# Fit
rand_search = RandomizedSearchCV(pipe, params, n_iter=30, cv=3, n_jobs=-1, verbose=1)
rand_search.fit(data["data"], data["target"])

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  5.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [49]:
rand_search.best_score_

0.8941083998626492

In [50]:
rand_search.best_params_

{'clf__n_estimators': 49,
 'lsi__svd__n_components': 78,
 'lsi__vect__max_df': 0.9253785862354514}

In [55]:
pipe.named_steps

{'lsi': Pipeline(memory=None,
          steps=[('vect',
                  TfidfVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.float64'>,
                                  encoding='utf-8', input='content',
                                  lowercase=True, max_df=1.0, max_features=None,
                                  min_df=1, ngram_range=(1, 2), norm='l2',
                                  preprocessor=None, smooth_idf=True,
                                  stop_words='english', strip_accents=None,
                                  sublinear_tf=False,
                                  token_pattern='(?u)\\b\\w\\w+\\b',
                                  tokenizer=None, use_idf=True,
                                  vocabulary=None)),
                 ('svd',
                  TruncatedSVD(algorithm='randomized', n_components=100,
                               n_iter=10, random

In [56]:
pipe.named_steps["lsi"].named_steps["vect"]

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [58]:
best_pipeline = rand_search.best_estimator_

In [61]:
data["data"][1]

' ajteel@dendrite.cs.Colorado.EDU (A.J. Teel) writes...\n\nOn whose authority do you have this and on what grounds was it \ndismissed?\n\n\t\t\t\t\t\tDaniel Reitman\n\nHOW NOT TO WRITE A DEED\n\nOne case involved the construction of a conveyance to grantees "jointly, as \ntenants in common, with equal rights and interest in said land, and to the \nsurvivor thereof, in fee simple. . . . To Have and to Hold the same unto the \nsaid parties hereto, equally, jointly, as tenants in common, with equal rights \nand interest for the period or term of their lives, and to the survivor thereof \nat the death of the other."'

In [60]:
best_pipeline.named_steps["lsi"].named_steps["vect"].transform([data["data"][0]]).todense().shape

(1, 123386)

In [62]:
rand_search.best_params_

{'clf__n_estimators': 49,
 'lsi__svd__n_components': 78,
 'lsi__vect__max_df': 0.9253785862354514}

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [64]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [65]:
doc = nlp("Two bananas in pyjamas")

In [66]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [67]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [69]:
X = get_word_vectors(data.data)

len(X) == len(data.data)

True

In [71]:
len(X)

1058

In [72]:
X[0].shape

(300,)

In [73]:
rfc.fit(X, data["target"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [74]:
rfc.score(X, data["target"])

0.9877126654064272

In [ ]:
from multiprocessing import Pool, cpu_count

In [ ]:
pool = Pool(cpu_count())

In [ ]:
def to_vec(doc):
    return nlp(doc).vector

In [ ]:
%%time
X = pool.map(to_vec, data["data"])

## Follow Along

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle competition
* Find another text classification task to work on